##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Basi di customizzazione: tensori e operazioni

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/customization/basics"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Visualizza su TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/it/tutorials/customization/basics.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />segui in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/it/tutorials/customization/basics.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Visualizza il sorgente su GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/it/tutorials/customization/basics.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Scarica il notebook</a>
  </td>
</table>

Note: La nostra comunità di Tensorflow ha tradotto questi documenti. Poichè queste traduzioni sono *best-effort*, non è garantito che rispecchino in maniera precisa e aggiornata la [documentazione ufficiale in inglese](https://www.tensorflow.org/?hl=en). 
Se avete suggerimenti per migliorare questa traduzione, mandate per favore una pull request al repository Github [tensorflow/docs](https://github.com/tensorflow/docs). 
Per proporsi come volontari alla scrittura o alla review delle traduzioni della comunità contattate la 
[mailing list docs@tensorflow.org](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs).

Questo è un tutorial introduttivo su TensorFlow che mostra come:

* Importare i pacchetti richiesti
* Creare e usare tensori
* Usare l'accelerazione GPU
* Dare una dimostrazione del `tf.data.Dataset`

In [0]:
from __future__ import absolute_import, division, print_function

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


## Importare TensorFlow

Per iniziare, importa il modulo `tensorflow`. A partire da TensorFlow 2.0, l'esecuzione eager è attiva di default. Questo permette un frontend più interattivo verso TensorFlow, sui quali dettagli si discuterà oltre.

In [0]:
import tensorflow as tf

## Tensori

Un Tensore è un array multi-dimensionale. Similmente all'oggetto `ndarray` di NumPy, gli oggetti `tf.Tensor` hanno un tipo di dato e una forma. Inoltre, gli oggetti `tf.Tensor` possono risiedere in un acceleratore di memoria (come una GPU). TensorFlow offre una libreria ricca di operazioni ([tf.add](https://www.tensorflow.org/api_docs/python/tf/add), [tf.matmul](https://www.tensorflow.org/api_docs/python/tf/matmul), [tf.linalg.inv](https://www.tensorflow.org/api_docs/python/tf/linalg/inv) etc.) che consumano e producono oggetti `tf.Tensor`. Queste operazioni convertono automaticamente tipi nativi di Python, ad esempio:


In [0]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

Ogni oggetto `tf.Tensor` ha una forma e un tipo di dato:

In [0]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

Le differenze più ovvie tra gli array NumPy e i `tf.Tensor` sono:

1. I tensori possono essere supportati da acceleratori di memoria (come GPU, TPU).
2. I tensori sono immutabili.

### Compatibilità con NumPy 

La conversione tra oggetti `tf.Tensor` di TensorFlow e un `ndarray` di NumPy è facile:

* Le operazioni di TensorFlow convertono automaticamente gli ndarrays di NumPy a tensori.
* Le operazioni di NumPy convertono automaticamente i tensors a ndarrays di NumPy.appresentazione di memoria, se possible
I tensori sono convertiti esplicitamente ad ndarrays di NumPy usando il loro metodo. Queste conversioni sono tipicamente economiche dal momento che l'array e `tf.Tensor` condividono la stessa rappresentazione di memoria, se possible. Tuttavia, condividere la rappresentazione di memoria sottostante non è sempre possibile dal momento che `tf.Tensor` può essere hostata nella memoria GPU mentre gli array NumPy sono sempre supportati dalla memoria, e la conversazione coinvolge una copia dalla GPU e mentre gli array numPy sono sempre protetti dalla memoria.

In [0]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

## Accelerazione GPU

Molte operazione di TensorFlow sono accelerate usando la GPU per la computazione. In assenza di annotazioni, TensorFlow decide automaticamente se usare la GPU o la CPU per un'operazione—copiando il tensore tra la memoria CPU e quella della GPU se necessario. I Tensori prodotti da un'operazione sono tipicamente supportate dalla memoria del device su cui l'operazione ha eseguito, per esempio:

In [0]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.config.experimental.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

### Nomi dei Device

La proprietà `Tensor.device` fornisce una string interamente qualificata con il nome del device che ospita il contenuto del tensore. Questo nome codifica molti dettagli, tra cui un identificativo dell'indirizzo di rete del host su cui questo programma sta eseguendo e il device dentro quel host. Questo è richiesto per l'esecuzione distribuita di un programma TensorFlow. La stringa termina con `GPU:<N>` se il tensore è piazzato nel `N`-esimo GPU del host.



### Placement esplicito su un Device

In TensorFlow, il termine *placement* si refersce a come operazioni individuali sono assegnate a un device per l'esecuzione. Come menzionato, quando non c'è una guida esplicita fornita, TensorFlow decide automaticamente su quale device eseguire un'operazione e copia il tensore su quel device se necessario. Tuttavia le operazione di TensorFlow possono essere esplicitamente assegnati a device specifici usando il context manager `tf.device`, per esempio:

In [0]:
import time

def time_matmul(x):
  start = time.time()
  for loop in range(10):
    tf.matmul(x, x)

  result = time.time()-start

  print("10 loops: {:0.2f}ms".format(1000*result))

# Force execution on CPU
print("On CPU:")
with tf.device("CPU:0"):
  x = tf.random.uniform([1000, 1000])
  assert x.device.endswith("CPU:0")
  time_matmul(x)

# Force execution on GPU #0 if available
if tf.config.experimental.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"): # Or GPU:1 for the 2nd GPU, GPU:2 for the 3rd etc.
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("GPU:0")
    time_matmul(x)

## Datasets

Questa sezione usa la [API `tf.data.Dataset`](https://www.tensorflow.org/guide/datasets) per costruire una pipeline per fornire i dati al tuo modello. L'API `tf.data.Dataset` è usata per costruire pipeline di input complesse e performanti a partire da pezzi semplici e riutilizzabili che saranno dati in pasto ai loop di addestramento o valutazione del tuo modello.

### Creare un `Dataset` sorgente

Crea un dataset *sorgente* usando una delle funzioni factory come [`Dataset.from_tensors`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensors), [`Dataset.from_tensor_slices`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices), o usando oggetti che leggono da file come [`TextLineDataset`](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset) o [`TFRecordDataset`](https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset). Vedi la [guida dei TensorFlow Dataset](https://www.tensorflow.org/guide/datasets#reading_input_data) per ulteriori informazioni.

In [0]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

# Create a CSV file
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

### Applicare trasformazioni

Usa le funzioni di trasformazione come [`map`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map), [`batch`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch), e [`shuffle`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle) per applicare trasformazioni sui record del dataset.

In [0]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

### Iterare

Gli oggetti `tf.data.Dataset` supportano le iterazioni per ciclare sui record:

In [0]:
print('Elements of ds_tensors:')
for x in ds_tensors:
  print(x)

print('\nElements in ds_file:')
for x in ds_file:
  print(x)